In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

import pandas as pd
from datetime import datetime

In [3]:
aylar = {
    "Ocak": "01",
    "Şubat": "02",
    "Mart": "03",
    "Nisan": "04",
    "Mayıs": "05",
    "Haziran": "06",
    "Temmuz": "07",
    "Ağustos": "08",
    "Eylül": "09",
    "Ekim": "10",
    "Kasım": "11",
    "Aralık": "12"
}

In [6]:
def get_details_new(element, cmb):
    year = int(element.find_all("td", {"class": "searchResultsAttributeValue"})[0].text)
    mileage = int(element.find_all("td", {"class": "searchResultsAttributeValue"})[1].text.replace(".", ""))
    color = element.find_all("td", {"class": "searchResultsAttributeValue"})[2].text.strip()
    price = int(element.find("td", {"class": "searchResultsPriceValue"}).text.strip()[:-3].replace(".", ""))
    p1 ,_ ,p3 = element.find("td", {"class": "searchResultsDateValue"}).text.strip().split("\n")
    p1, p2 = p1.split(" ")
    p2 = aylar[p2]
    final_date = datetime.strptime(f"{p1}-{p2}-{p3}", "%d-%m-%Y")
    listing_age = (datetime.now() - final_date).days
    location = element.find("td", {"class": "searchResultsLocationValue"}).get_text(separator=" ").strip().split(" ")[0]
    fuel = cmb[0]
    transmission = cmb[1]

    model = element.find("td", {"class": "searchResultsTagAttributeValue"}).text.strip()
    # link = "https://www.sahibinden.com/" + element.find("a", {"class": "classifiedTitle"}).get("href")
    
    return {
        "model": model,
        "age": datetime.now().year - year,
        "mileage": mileage,
        "color": color,
        "price": price,
        "listing_age": listing_age,
        "location": location,
        "fuel": fuel,
        "transmission": transmission
    }

In [7]:
fuel = ["benzin", "benzin-lpg", "dizel"]
transmission = ["manuel", "otomatik"]

In [8]:
all_combinations = []
for i in fuel:
    for j in transmission:
        all_combinations.append((i, j))

In [9]:
info = []

In [10]:
model_info = ["bmw", "3-serisi", "320i"]
model_name = "-".join(model_info)

In [11]:
for cmb in all_combinations:
    for i in range(1, 2):
        driver=webdriver.Edge()
        driver.maximize_window()
        driver.get(f"https://www.sahibinden.com/{model_name}/{cmb[0]}/{cmb[1]}?pagingOffset={(i-1)*50}&pagingSize=50")

        soup = BeautifulSoup(driver.page_source,'html.parser')
        all_elements = soup.find("tbody", {"class": "searchResultsRowClass"})
        
        if all_elements:
            individual_elements = all_elements.find_all("tr", {"class": "searchResultsItem"})
            
            for item in individual_elements:
                try:
                    info.append(get_details_new(item, cmb))
                except IndexError:
                    pass
            
            driver.quit()
        else:
            driver.quit()
            break

In [12]:
data = pd.DataFrame(info)

In [13]:
data.to_csv("new_dataset.csv", index=False)

In [14]:
data

,model,age,mileage,color,price,listing_age,location,fuel,transmission
0,320i,32,337000,Siyah,485000,6,İstanbul,benzin,manuel
1,320i,27,202000,Gri,625000,0,İstanbul,benzin,manuel
2,320i,27,336500,Lacivert,370000,0,Konya,benzin,manuel
3,320i Cabrio,27,370000,Kırmızı,920000,0,Kayseri,benzin,manuel
4,320i,27,250000,Mavi,930000,1,İstanbul,benzin,manuel
...,...,...,...,...,...,...,...,...,...
198,320i,25,330000,Beyaz,600000,4,Trabzon,benzin-lpg,otomatik
199,320i,25,256000,Siyah,525000,2,Burdur,benzin-lpg,otomatik
200,320i Premium,15,296000,Siyah,670000,2,Adana,benzin-lpg,otomatik
201,320i,17,140000,Gri,854999,2,Antalya,benzin-lpg,otomatik
